<a href="https://colab.research.google.com/github/SinaRampe/kb_download/blob/main/cpic_recommendations_publications_download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2 -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.7 MB/s eta 0:00:00


In [ ]:
import requests
import pandas as pd
import os
import re
from PyPDF2 import PdfReader, PdfWriter

In [ ]:
cpic_recommendations_url = "https://api.cpicpgx.org/v1/recommendation"

def fetch_cpic_recommendations():

    response = requests.get(cpic_recommendations_url)
    data = response.json()

    return data

In [ ]:
recommendations = fetch_cpic_recommendations()

In [ ]:
df = pd.DataFrame(recommendations)

In [ ]:
df.shape

(2098, 16)

In [ ]:
df.head(2)

,id,guidelineid,drugid,implications,drugrecommendation,classification,phenotypes,activityscore,allelestatus,lookupkey,population,comments,version,dosinginformation,alternatedrugavailable,otherprescribingguidance
0,3369605,100421,RxNorm:190521,{'HLA-B': 'Low or reduced risk of abacavir hyp...,Use abacavir per standard dosing guidelines,Strong,{},{},{'HLA-B': 'HLA-B*57:01 negative'},{'HLA-B': '*57:01 negative'},general,n/a,1,False,False,False
1,3369606,100421,RxNorm:190521,{'HLA-B': 'Significantly increased risk of aba...,Abacavir is not recommended,Strong,{},{},{'HLA-B': 'HLA-B*57:01 positive'},{'HLA-B': '*57:01 positive'},general,n/a,1,False,False,False


In [ ]:
unique_guidelineids = df['guidelineid'].unique().tolist()

len(unique_guidelineids)

24

In [ ]:
unique_guidelineids[0]

100421

In [ ]:
cpic_guidelines_url = "https://api.cpicpgx.org/v1/publication?guidelineid=eq."

def download_pdf(url, filename):
    response = requests.get(url)
    folder_path = "./chic_recommendations_publications"
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    full_path = os.path.join(folder_path, filename)
    with open(full_path, 'wb') as f:
        f.write(response.content)
    return full_path

def add_metadata_to_pdf(path, metadata):
    reader = PdfReader(path)
    writer = PdfWriter()
    for page in reader.pages:
        writer.add_page(page)
    writer.add_metadata(metadata)
    with open(path, "wb") as output_pdf:
        writer.write(output_pdf)

def fetch_cpic_guideline(guideline_id):
    urls = []
    response = requests.get(f"{cpic_guidelines_url}{guideline_id}")
    data = response.json()

    for item in data:
        url = item['fulltextfile']
        if url is not None:
            urls.append(url)
            safe_title = re.sub(r'[\\/*?:"<>|]', "", item['title'])
            filename = f"{item['year']}_{safe_title}.pdf"
            path = download_pdf(url, filename)
            metadata = {
                '/Title': item['title'],
                '/Author': ', '.join(item['authors']),
                '/Subject': item['url'],
                '/Keywords': f"Year: {item['year']}, PMID: {item['pmid']}"
            }
            add_metadata_to_pdf(path, metadata)

In [ ]:
for i in unique_guidelineids:
  fetch_cpic_guideline(i)

In [ ]:
!zip -r chic_recommendations_publications.zip chic_recommendations_publications/

  adding: chic_recommendations_publications/ (stored 0%)
  adding: chic_recommendations_publications/2016_Clinical Pharmacogenetics Implementation Consortium (CPIC) Guideline for CYP2D6 Genotype and Use of Ondansetron and Tropisetron.pdf (deflated 13%)
  adding: chic_recommendations_publications/2014_Clinical Pharmacogenetics Implementation Consortium (CPIC) Guidelines for CYP2C9 and HLA-B Genotype and Phenytoin Dosing.pdf (deflated 19%)
  adding: chic_recommendations_publications/2014_Clinical Pharmacogenetics Implementation Consortium (CPIC) guidelines for cytochrome P450 2D6 (CYP2D6) genotype and codeine therapy 2014 Update.pdf (deflated 29%)
  adding: chic_recommendations_publications/2020_Clinical Pharmacogenetics Implementation Consortium (CPIC) Guideline for CYP2C19 and Proton Pump Inhibitor Dosing..pdf (deflated 13%)
  adding: chic_recommendations_publications/2017_Clinical Pharmacogenetics Implementation Consortium (CPIC) Guideline for Dihydropyrimidine Dehydrogenase Genotype 